# 1. Learning Deep Features via Congenerous Cosine Loss for Person Recognition  

## 1.1 Abstract

人物识别目的是在复杂的场景中识别跨越时间和空间的相同身份。在本文中提出了一种新方法，通过训练一个网络来获得鲁棒且较好的特征来解决这个问题。我们直接比较和优化了两个特征之间的余弦距离 - 扩大类差异并减少了类内的差异。我们通过最小化样本与它们的质心之间的余弦距离来提出一种连续的余弦损失。这样的设计降低了复杂度，并且可以通过使用标准化输入的 softmax 来实现。我们的方法也不同于以前的人物识别工作，我们不对测试子集进行额外的训练。通过测量测试集中几个身体区域的相似性来确定人的身份。实验结果表明，所提出的方法比之前 state-of-the-arts 的方法有着更高的准确率。  

## 1.2 Introduction  

随着智能手机和数码相机需求的不断增长，今天的人们会拍摄更多照片来记录日常生活和故事。这种趋势迫切需要智能工具从个人数据，社交媒体或互联网上获取数以千计的图像中跨越不同的时间和空间来识别同一个人。以前的工作已经证明，由于许多因素，诸如非正面，不同的光照，外观的可变性，身份纹理等，在这种不受约束的环境中人物识别仍然是一个具有挑战性的问题。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/person/1.jpg?raw=true)  

最近提出的 PIPA 数据集包含数千幅具有复杂场景和相似外观的图像。图像的亮度，大小和上下文变化很大，许多实例都有部分甚至没有面部。图 1 显示了来自训练和测试集的一些样本。以前的工作通过多线索，多层次的方式来识别同一个人，其中训练集仅用于提取特征并且后续分类器（SVM或神经网络）在 test 0 中被训练。识别系统在 test 1 组中进行评估。我们认为这样的做法在实际应用中是不可行的，因为第二次 test 0 的训练是附加的并且如果添加新样本需要重新训练。相反，我们的目标是提供一组鲁棒的，广义的特征表示，它们直接在训练集上训练，并通过测量测试集之间的特征相似度来识别人物。没有必要在测试 test 0 上进行训练，即使有新的数据进入，系统仍然运行良好。  

如图 1 底部所示，green box 中的女人与 light green box 中的女人戴着类似的围巾，她的脸部部分出现或者在某些情况下脸部不会出现。为了获得鲁棒的特征表示，我们针对不同的区域训练几个不同深度模型，并结合不同区域特征的相似度得分来预测他们的身份。我们的重要发现是，在训练期间，交叉熵损失不能保证类别内样本的相似性。它放大了不同类别之间的差异，并忽略了同一类别的特征相似性。为此，我们提出了一个连续的余弦损失，即 COCO，以扩大类间差异，并缩小类内差异。它通过测量样本与其集群质心之间的余弦距离来实现。此外，我们还将每个 region patch 与预定义的基本位置对齐，以进一步使特征空间中的类别内的样本更加接近。这种对齐策略也可能使网络不太容易过拟合。  

图 2 显示了我们提出的算法训练的 pipeline。图像中的每一个实例都用一个 ground truth head 标注，我们使用 RPN 框架分别训练一个脸部和人体检测器。然后使用人体姿势估计器来检测人的关键部位以定位上身区域。crop 四个区域后，我们进行仿射变换，将不同的 patch 从训练样本对齐到“基本”位置。分别在 PIPA 训练集上使用 COCO 损失训练四个深度模型来获取一系列鲁棒的特征。总而言之，这项工作的贡献如下：  

- 提出一个连续余弦损失来直接优化类别内部和跨类别样本间的余弦距离。它是以简单的 softmax 实现的，具有较低的复杂度。
- 设计了一个人体识别 pipeline，利用多个身体区域获得可区分的特征表达，而无需对测试集进行二次训练。
- 通过仿射变换将区域 patch 与基准位置对齐，以减少样本间的差异，使网络不易过拟合。  

## 1.3 Related Work  

基于图像的人物识别旨在识别日常生活照片中人物的身份，这些问题可能会因杂乱的背景而变得复杂。Anguelov 等首先通过提出一个马尔科夫随机场框架来解决这个问题，以结合所有情境提示来识别人的身份。最近，Zhang 等人为这项任务引入了一个大规模数据集。他们积累了由深层模型训练的 poselet 级别人物识别器的线索，以补偿姿势变化。Li 等在 LSTM 中嵌入场景和关系上下文，并将人的识别作为序列预测任务。  

人物重新识别是为了匹配摄像机中不同视角下的行人图像，并在视频中引发了许多重要的应用。现有的工作采用度量学习和中级特征学习来解决这个问题。 Yi 等用 Siamese 深度网络来学习图像对之间的相似性。人物识别与重新识别的主要区别在于数据流。前者是在不同的地点和时间识别同一个人。在大多数情况下，在不同场合中人物的外观会有很大的变化。后者是在连续的视频中检测人，这意味着外观和背景随着时间并没有很大的变化。  

深层神经网络近年来很大程度地推进了计算机视觉领域的发展，提高了图像分类，目标检测和目标跟踪等任务的性能。深度学习成功背后的精髓在于其在高维空间中的非线性复杂性的优越表现力和大规模数据集，深层网络可以在很大程度上学习复杂的模式和代表性特征。  

## 1.4 Algorithm  

所提出的 COCO 损失是针对每个身体区域进行训练以获得鲁棒的特征。  

### 1.4.1 Region detection  

利用四个区域 $r \in {1,...,4}$ 即脸部，头部，全身和上身来训练特征。先来介绍这些区域的检测。

Face: 先用 Faster RCNN 中的 RPN 预训练一个人脸检测器。数据为来自互联网的大约为300,000 张图像。网络结构是浅层 ResNet 模型，移除了 res_3b 之后的层并添加两个损失层（分类和回归）。然后，我们对 PIPA 训练集中的人脸采用 COCO 损失对模型进行微调。人脸检测器识别脸部的 m 个关键点（眼睛，眉毛，嘴巴等），并通过平移，旋转和缩放将检测到的脸部 patch 进行标准化。设 $p,q \in R^{m*2}$ 分别表示人脸模型检测的 m 个关键点和对齐结果。我们将 $P,Q$ 定义为两个仿射空间，然后仿射变换 $A:P \mapsto Q$ 定义为：  

$p \mapsto q = Ap + b$

其中 $A \in R^{m*m}$ 是 P 中的线性变换矩阵，$b \in R^{m*2}$是 Q 中的偏差。这样的对齐方案可以确保类别内和类别之间的样本没有很大的差异：如果网络是在没有对齐的情况下学习的，它必须区分更多的模式，例如人员之间不同的旋转，更容易出现过拟合；如果数据进行了对其，则可以更侧重于区分不同身份的功能，而不用管旋转，视角等。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/person/2.jpg?raw=true) 

Head, whole body and upper body：头部区域是作为每个人的 ground truth 给出的。为了检测整个身体，我们还在 RPN 框架中预先训练了一个检测器。 该模型在大型人类数据集上进行了训练，我们使用 4638717 副图像中的 87021 个人进行训练。网络结构是一个 inception 模型，最终的 pooling 被完全连接层所取代。为了确定上身区域，我们进行了人体姿态估计以识别身体的关键点，并由此确定上半身的位置。头部，全身和上半身的检测器基于 PIPA 训练集用 COCO loss 进行了微调，并做了面部部区域所述的类似的 patch 对准过程。图 2（c）显示了四个区域的对齐 patch。  

### 1.4.2  Congenerous cosine loss for training  

设计 COCO 损失是为了直接比较和优化两个特征之间的余弦距离。令 $f^{i,f} \in R^{D*1}$ 表示第 i 个样本在区域 r 的特征向量，其中 D 是特征维数。为简洁起见，我们去掉上标 r，因为每个区域模型都经历了相同的 COCO 训练。我们首先定义一个小批次 $B$ 中两个特征的余弦相似性，如下所示：  

$C(f^{(i)},f^{(j)}) = \frac {f^{(i)}*f^{(j)}} {\parallel f^{(i)} \parallel \parallel f^{(j)} \parallel} $

余弦相似度衡量两个样本在特征空间中的距离。理想的损失是增加类内样本的相似度，并扩大类间质心的距离。令 $l_i,l_j \in {1,...,K}$ 为样本 $i,j$ 的标签，其中 K 为类别总数，我们最大化下面的损失：  

$ L^{naive} = \sum_{i,j \in B} \frac {\delta (l_i,l_j) C(f^{(i)},f^{(j)})}{1 - \delta (l_i,l_j) C(f^{(i)},f^{(j)}) + \epsilon}$

其中$\delta(.,.)$ 是一个指示函数。这样的设计在理论上是合理的，但仍然存在计算效率低下的问题。此外，如果我们直接计算来自小批次的两个任意样本的损失，网络会遭受不稳定的参数更新，并且很难收敛。  

受中心损失的启发，我们将类别 k 的质心定义为小样本 B 上特征的平均值：  

$c_k = \frac {\sum_{i \in B} \delta (l_i,k) f^{(i)}}{\sum_{i \in B} \delta (l_i,k) + \epsilon} \in R^{D*1}$

结合上面两个公式，我们有以下样本 i 的输出需要去最大化：  

$p_{l_i}^{(i)} = \frac {exp C(f^{(i)},c_{l_i})}{\sum_{k \neq l_i} exp C(f^{(i)},c_k)} \in R$

公式通过类别质心来测量一个样本与其他样本的距离，而不是上面一样直接进行两两比较。确保样本 i 与其自己的类别 $l_i$ 足够接近。指数运算是将余弦相似度转换为归一化概率输出，范围从 0 到 1。  

为此，我们提出了cogenerous cosine（COCO）损失，这是为了增加类内的相似性，并以合作的方式扩大跨类别的变化：  

$ L^{COCO} = \sum_{i \in B} L^{(i)}  = -\sum_{k,i} t_k^{(i)} log p_k^{(i)} = - \sum_{i \in B} log p_{l_i}^{(i)}$

在实践中，可以通过 softmax 操作以一种简洁的方式实施 COCO loss：  

$p_{k}^{(i)} = \frac {exp (\hat{c}_k^T \cdot \hat{f}^{(i)})}{\sum_m exp (\hat{c}_m^T \cdot \hat{f}^{(i)})} = softmax(z_k^{(i)})$

其中 $z_k^{(i)} = \hat{c}_k^T \cdot \hat{f}^{(i)}$ 是softmax的输入。 $\hat{c}_k$ 可以看作是分类层中的权重，其中偏差项为零。  

请注意，这些特征和集群质心都是端对端训练的。这些特征是从预训练模型初始化的，并且 $c_k$ 的初始值是通过公式计算的。  

### 1.4.3  Relationship of COCO with counterparts  

COCO 损失被表述为特征空间中的度量学习方法，其使用余弦距离中的集群质心作为度量来扩大类间变化以及缩小类内差异。它可以通过几个限制条件下的softmax 操作来实现。图 3 显示了不同损失方案下的特征集群的可视化。对于 softmax 损失，它只强制跨类别的样本很远，而忽略一个类中的相似性。在COCO 损失中，我们用 softmax 之前的分类层来替换权重，并且有明确定义和可学习的集群质心。中心损失与我们的某些方面相似，但是，它需要一个外部存储器来存储类的中心，因此计算量是我们的两倍。Liu 提出了一个广义的 large-margin 损失，它也通过强化类内紧致性和类间可分性来学习判别性特征。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/person/3.jpg?raw=true) 

### 1.4.4   Inference  

在测试阶段，我们测量两个测试分片之间的特征的相似性，以基于 test_0 中的标签来识别 test_1 中的每个实例的身份。test_1 和 test_0 中的两个分片 i 和 j 之间的相似性由 $s_{i,j}^(r) = C(f^{(i,r)},f^{(j,r)})$，其中 r 表示特定的区域模型。关键问题是如何合并来自不同区域的相似度分数。 我们首先对初步结果$s^(r)$进行归一化处理，以使不同区域的分数具有可比性：  

$\hat{s}_{i,j}^{(r)} = (1 + exp [-(\beta_0 + \beta_1 s_{i,j}^{(r)})])^{-1}$

其中 $\beta_0, \beta_1$ 是逻辑回归的参数。最终得分 $S_{i,j}$ 是每个区域的归一化得分 $s_{ij}^{(r)}$ 的加权平均值。test_1 中的 patch i的身份由与参考集中的最大分数相对应的标签决定：$l_i = arg max_{j*} S_{i,j}$。 这样的方案保证了当新的训练数据被添加到 test_0 中时，不需要在测试集上训练第二个模型或SVM，这与以前的工作非常不同。

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/classification/person/4.jpg?raw=true) 

# 2. Rethinking Feature Discrimination and Polymerization for Large-scale Recognition  

# 3. Person Recognition in Social Media Photos

https://arxiv.org/abs/1702.06890